In [ ]:
#in this script, we implement our algorithm for matrix learning using different regularizers.
# In order to run these algorithms, Mosek has to be installed and the user must posses a valid Mosek licence. 

In [2]:
using JuMP 
using Mosek

<h2>Spectral learning<h2>

In [ ]:

# @param:weights : weights of the observed matrices. Sum of weights must be 1. and non-negative. the size is equal to the amount
#of observed graphs.
#@param: delta : size of the ball of the Wasserstein metric. delta > 0.
#@param: A : cluster form of the data. A entries must be 0 or 1. 
#@param: B : vector of matrices containing the observations of the graph B. If X,Y are matrices, an array of matrices
#is of the form Array[X,Y].
#@param: n : the dimension of the matrices.
#@param:N : number of observations.
#The set over which the is problem is optimized is:

In [ ]:
#display("text/latex",""" El conjunto sobre el que optimizamos es:
#\$ O = \\{\\lambda,S_1,..,S_N,Z^{1},..,Z^{N},C^{1},...,C^{N},W,W_1,W_2 \\}\\subseteq \\mathbb{R}\\times \\mathbb{R}^N
#\\times Sim^2(\\mathbb{R}^n)^N \\times Sim^2(\\mathbb{R}^n)^N \\times Sim^2(\\mathbb{R}^)^3
#\$""")

In [22]:
#Computes the error of estimating the cluster structure of the observations by the matrix A.
function learnSpectralError(weights,delta,A,B,n)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
     @variable(m,S[1:N])
    @variable(m, W[1:n,1:n], Symmetric)
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    C = @variable(m, [1:n, 1:n], Symmetric)
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    #Constraints for Z
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constraints for C 
    @constraint(m,C.>=-(2*A-one*one'-W))
    @constraint(m,C.>=0)
    [@constraint(m, one'*C*one<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Constraints for W
    @SDconstraint(m,[W1 W;W W2]>=0)
    @constraint(m,trace(W1)+trace(W2)<=2*lambda)

    #Objective function, calls the function prod
    @objective(m,Min,lambda*delta+ dot(weights,S+produ.(Z,n)))
    status = solve(m)
    return([getvalue(C),getvalue(lambda),getvalue(S),getobjectivevalue(m)])
end


learnError (generic function with 1 method)

In [2]:
#Deprecated. use instead  learnEspectralGraph
#Finds the matrix A which minimices the expected error of cluster structure. This
#time, A is a simmtric entrywise-positive matrix.
#The parameters are the same as in learnError function
function learnGraph(weights,delta,B,n)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
     @variable(m,S[1:N])
    @variable(m, W[1:n,1:n], Symmetric)
    @variable(m,A[1:n,1:n], Symmetric)
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    C = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    #Constraints for Z
    [@constraint(m, -Z[p][k,l] <= A[k,l]-B[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    [@constraint(m,  A[k,l]-B[p][k,l]<=Z[p][k,l]) for p in 1:N for k in 1:n for l in 1:n]
    #Constraints for C 
    [@constraint(m,C[p][k,l]>=0)for p in 1:N for k in 1:n for l in 1:n]
    [@constraint(m, produ(C[p],n)<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    [@constraint(m,-C[p][k,l]<=2*A[k,l]-1-W[k,l])for p in 1:N for k in 1:n for l in 1:n]
    #Constraints for W
    @SDconstraint(m,[W1 W;W W2]>=0)
    @constraint(m,trace(W1)+trace(W2)<=2*lambda)
    #constrains for A
    [@constraint(m,A[k,l]>=0) for k in 1:n for l in 1:n]
    [@constraint(m,A[k,l]<=1) for k in 1:n for l in 1:n]
    #Objective function, calls the function prod
    @objective(m,Min,lambda*delta+ dot(weights,S+produ.(Z,n)))
    status = solve(m)
    return([roundZeroMatrix(getvalue(A)),getvalue(S),getvalue(lambda),roundZeroMatrix(getvalue(W)),getobjectivevalue(m)])
end

learnGraph (generic function with 1 method)

In [ ]:
#Finds the matrix A which minimices the expected error of cluster structure. This
#time, A is a simmtric entrywise-positive matrix.
#The parameters are the same as in learnError function
function learnEspectralGraph(weights,delta,B,n)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
     @variable(m,S[1:N])
    @variable(m, W[1:n,1:n], Symmetric)
    @variable(m,A[1:n,1:n], Symmetric)
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    C = @variable(m, [1:n, 1:n], Symmetric)
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    #Constraints for Z
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constraints for C 
    @constraint(m,C.>=-(2*A-one*one'-W))
    @constraint(m,C.>=0)
    [@constraint(m, one'*C*one<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Constraints for W
    @SDconstraint(m,[W1 W;W W2]>=0)
    @constraint(m,trace(W1)+trace(W2)<=2*lambda)
    #constrains for A
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    #Objective function, calls the function prod
    @objective(m,Min,lambda*delta+ dot(weights,S+produ.(Z,n)))
    status = solve(m)
    return([getvalue(A),getvalue(S),getvalue(lambda),getvalue(W),getobjectivevalue(m)])
end

<h2>L2 learning<h2>

In [1]:
#Finds the matrix A which minimices the expected error of cluster structure. This
#time, A is a simmtric entrywise-positive matrix.
#The parameters are the same as in learnError function
function learnl2Graph(weights,delta,B,n)
 N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    @variable(m,A[1:n,1:n], Symmetric)
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n], Symmetric)
    #Q es una variable auxiliar, juega el papel de Lambda en la demostracion
    @variable(m,Q[1:n,1:n])
    # u es una variable auxiliar para calcular la norma infinito de W.
   # @variable(m,u>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
 
    #Constrains...
    #constrains for lambda already in the definition of the varible.
    #Constrains for A.
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    #Constrains for Z.
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constrains for W. 
    @constraint(m,vec(W)'vec(W)<=lambda)
     #Constrains for Q.
    @constraint(m,Q .>=-(2*A-one*one'-W))
    @constraint(m,Q .>=0)
    [@constraint(m, one'*Q*one<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+produ.(Z,n)))
    #TT = STDOUT # save original STDOUT stream
    #redirect_stdout()
    solve(m)
    #redirect_stdout(TT) # restore STDOUT
    return([getvalue(A),getvalue(S),getvalue(lambda),getvalue(W),getobjectivevalue(m)])
end

LoadError: [91mUndefVarError: @variable not defined[39m

In [ ]:
function learnl2Error(weights,delta,A,B,n)
 N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n], Symmetric)
    #Q es una variable auxiliar, juega el papel de Lambda en la demostracion
    @variable(m,Q[1:n,1:n])
    # u es una variable auxiliar para calcular la norma infinito de W.
   # @variable(m,u>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
 
    #Constrains...
    #constrains for lambda already in the definition of the varible.

    #Constrains for Z.
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constrains for W. 
    @constraint(m,vec(W)'vec(W)<=lambda)
     #Constrains for Q.
    @constraint(m,Q .>=-(2*A-one*one'-W))
    @constraint(m,Q .>=0)
    [@constraint(m, one'*Q*one<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+produ.(Z,n)))
    #TT = STDOUT # save original STDOUT stream
    #redirect_stdout()
    solve(m)
    #redirect_stdout(TT) # restore STDOUT
    return([getvalue(Q),getvalue(lambda),getvalue(S),getobjectivevalue(m)])
end

<h2>Linear learning<h2>

In [ ]:
# @param:weights : weights of the observed matrices. Sum of weights must be 1. and non-negative. the size is equal to the amount
#of observed graphs.
#@param: delta : size of the ball of the Wasserstein metric. delta > 0.
#@param: A : cluster form of the data. A entries must be 0 or 1. 
#@param: B : vector of matrices containing the observations of the graph B. If X,Y are matrices, an array of matrices
#is of the form Array[X,Y].
#@param: n : the dimension of the matrices.
#@param:N : number of observations.
#The set over which the is problem is optimized is:

In [ ]:
function learnLinearGraph(weights,delta,B,n)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    @variable(m,A[1:n,1:n], Symmetric)
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n],Symmetric)
    #Q es una variable auxiliar, juega el papel de Lambda en la demostracion
    @variable(m,Q[1:n,1:n],Symmetric)
    # u es una variable auxiliar para calcular la norma infinito de W.
    @variable(m,u>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
 
    #Constrains...
    #constrains for lambda already in the definition of the varible.
    #Constrains for A.
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    #Constrains for Z.
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constrains for W. 
    @constraint(m,u<=lambda)
    @constraint(m,u .>=W)
    @constraint(m,u .>=-W)
     #Constrains for Q.
    @constraint(m,Q .>=-(2*A-one*one'-W))
    @constraint(m,Q .>=0)
    [@constraint(m, one'*Q*one<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+produ.(Z,n)))
    #TT = STDOUT # save original STDOUT stream
    #redirect_stdout()
    solve(m)
    #redirect_stdout(TT) # restore STDOUT
    return([getvalue(A),getvalue(S),getvalue(lambda),getvalue(W),getobjectivevalue(m)])
end

In [ ]:
function learnLinearError(weights,delta,A,B,n)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n],Symmetric)
    #Q es una variable auxiliar, juega el papel de Lambda en la demostracion
    @variable(m,Q[1:n,1:n])
    # u es una variable auxiliar para calcular la norma infinito de W.
    @variable(m,u>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    #Constrains...
    #constrains for lambda already in the definition of the varible.
    #Constrains for Z.
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constrains for W. 
    @constraint(m,u<=lambda)
    @constraint(m,u .>=W)
    @constraint(m,u .>=-W)
    #Constrains for Q.
    @constraint(m,Q .>=-(2*A-one*one'-W))
    @constraint(m,Q .>=0)
    [@constraint(m, ones(n)'*Q*ones(n)<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+produ.(Z,n)))
    #TT = STDOUT # save original STDOUT stream
    #redirect_stdout()
    solve(m)
    #redirect_stdout(TT) # restore STDOUT
    return([getvalue(Q),getvalue(lambda),getvalue(S),getobjectivevalue(m)])
end

<h1>Regularization learning<h1>

In [ ]:
#Funcion para calcular el lado derecho de la desigualdad, i.e min A 
# 1/m (sum ||A-B_i||_1) +delta(||2A-11^t||_nuc)
#asumimos que los pesos son todos iguales a 1/N
function learnRegularized2A(weights,delta,B,n)
    N = length(B)
    m = Model(solver=MosekSolver())
    @variable(m,A[1:n,1:n], Symmetric)
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    @variable(m,W[1:n,1:n],Symmetric)
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    #Constraints for Z
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #constrains for A
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    @constraint(m,W.>=2*A-ones(n)*ones(n)')
    @constraint(m,W.<=2*A-ones(n)*ones(n)')
    @SDconstraint(m,[W1 W;W W2]>=0)
    @objective(m,Min, dot(weights,produ.(Z,n))+delta*(0.5*(trace(W1)+trace(W2))))
     status = solve(m)
    return([getvalue(A),getvalue(W),getvalue(dot(weights,produ.(Z,n))),getvalue(Z),getobjectivevalue(m)])
end
 

In [ ]:
#funcion para calcular #1/m (sum ||A-B_i||_1) +delta(||2A-11^t||_nuc)
function learnRegularized2AError(A,weights,delta,B,n)
    N = length(B)
    m = Model(solver=MosekSolver())
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    @variable(m,W[1:n,1:n],Symmetric)
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    #Constraints for Z
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    @constraint(m,W.>=2*A-ones(n)*ones(n)')
    @constraint(m,W.<=2*A-ones(n)*ones(n)')
    @SDconstraint(m,[W1 W;W W2]>=0)
    @objective(m,Min, dot(weights,produ.(Z,n))+delta*(0.5*(trace(W1)+trace(W2))))
     status = solve(m)
    return([getobjectivevalue(m),getvalue(W),getvalue(Z)])
end

In [ ]:
#Funcion para calcular el lado derecho de la desigualdad, i.e min A 
# 1/m (sum ||A-B_i||_1) +delta(||A||_nuc)
#asumimos que los pesos son todos iguales a 1/N
function learnRegularizedA(weights,delta,B,n)
    N = length(B)
    m = Model(solver=MosekSolver())
    @variable(m,A[1:n,1:n], Symmetric)
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    #Constraints for Z
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #constrains for A
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    @SDconstraint(m,[W1 A;A W2]>=0)
    @objective(m,Min, dot(weights,produ.(Z,n))+delta*(0.5*(trace(W1)+trace(W2))))
     status = solve(m)
    return([getvalue(A),getvalue(dot(weights,produ.(Z,n))),getvalue(Z),getobjectivevalue(m)])
end


In [ ]:
#Funcion para calcular el lado derecho de la desigualdad, i.e 
#funcion para calcular #1/m (sum ||A-B_i||_1) +delta(||A||_nuc)
#asumimos que los pesos son todos iguales a 1/N
function learnRegularizedA(weights,delta,B,n)
    N = length(B)
    m = Model(solver=MosekSolver())
    @variable(m,A[1:n,1:n], Symmetric)
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    #Constraints for Z
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #constrains for A
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    @SDconstraint(m,[W1 A;A W2]>=0)
    @objective(m,Min, dot(weights,produ.(Z,n))+delta*(0.5*(trace(W1)+trace(W2))))
     status = solve(m)
    return([getvalue(A),getvalue(dot(weights,produ.(Z,n))),getvalue(Z),getobjectivevalue(m)])
end